# Production Code

In [ ]:
import os
import re
import numpy as np
import pandas as pd

source_directory = "../surveys/plain_txt/"

## Extract

### Global Variables

In [ ]:
# initialize the list of products we are capturing
product_list = ["maíz y polenta", "arroz", "fideos", "sorgo", "harina y trigo",
                "maní", "miel", "legumbres", "yerba maté", "té", "café",
                "azúcar", "aceite", "agua", "carne", "pescado", "pan",
                "verduras y frutas", "lácteos", "condimentos", "sal",
                "productos de recolección", "insumos", "levadura", "grasa",
                "huevos", "vi-3:"]
translation_list = ["corn_and_polenta", "rice", "pasta", "sorghum",
                    "flour_and_wheat", "peanut", "honey", "legumes",
                    "yerba_mate", "tea", "coffee", "sugar", "oil", "water",
                    "meat", "fish", "bread", "vegetables_and_fruits", "dairy",
                    "condiments", "salt", "harvest_products", "supplies",
                    "yeast", "grease_or_fat", "eggs"]
# column headers not including species column
column_headers = ["has_bought", "quantity", "units", "periodicity_1",
                  "number_of_periods_1", "periodicity_2", "number_of_periods_2",
                  "minimum_unit_price_pesos", "maximum_unit_price_pesos"]

### Helper Functions

In [ ]:
def find_between(str_1, str_2, contents):
    regex_between = str_1 + r"([\w\W]*)" + str_2
    regex_search = re.findall(regex_between, contents)

    return regex_search

### Parsing Functions

In [ ]:
def parse_interviewee(contents):
    name_regex = r"nombre de la persona[^\n:]*:[\s]*([^\n]*)\n"
    name_search = re.findall(name_regex, contents)
    if len(name_search) == 1:
        name = name_search[0]
        if name == "3":
            name = "N/A"
    else:
        alt_name_regex = r"nombre de la persona[^\n\(]*\(\w* +([^\n]*)\n"
        alt_name_search = re.findall(alt_name_regex, contents)
        if len(alt_name_search) == 1:
            name = alt_name_search[0]
            if name == "3":
                name = "N/A"
        else:
            name = "UNENCOUNTERED FORMAT"

    name = name.strip()

    return name


def parse_date_range(section_search):
    date_range = ""

    if len(section_search) == 1:
        section_string = section_search[0]

        date_regex = r"([\w ]+\d+ *[-–] *[\w ]+\d+)"
        date_search = re.findall(date_regex, section_string)

        if len(date_search) == 1:
            date_range = date_search[0]
        else:
            date_range = "UNENCOUNTERED FORMAT"
    else:
        date_range = "UNENCOUNTERED FORMAT"

    return date_range


def parse_section_6_2(section_search, product_list,
                      translation_list, column_headers):
    section_6_2_data = []

    if len(section_search) == 1:
        section_string = section_search[0] + "vi-3:"

        for i in range(len(product_list)-1):
            item_dict = {}
            item_1 = product_list[i]
            item_2 = product_list[i+1]
            term = translation_list[i]

            row_search = find_between(item_1, item_2, section_string)

            item_dict["product"] = term

            # if product row found
            if len(row_search) == 1:
                row_string = row_search[0]
                row_list = row_string.split("\n")
                row_list_len = len(row_list)

                if row_list_len >= 20:
                    # skip odd indexes (1, 3, 5, 7)
                    # drop first two rows (start count at -2)
                    count = -2
                    if (row_list[-1 * count] == ""
                       and row_list[-1 * count + 2] != ""):
                        count = -4

                    for j in range(row_list_len):
                        if count > 16:
                            break
                        if count >= 0 and count % 2 == 0:
                            col = column_headers[count//2]
                            entry = row_list[j]
                            entry = entry.strip()
                            item_dict[col] = entry
                        count += 1

                else:
                    for col in column_headers:
                        item_dict[col] = "UNENCOUNTERED FORMAT"
            else:
                for col in column_headers:
                    item_dict[col] = "UNENCOUNTERED FORMAT"

            section_6_2_data.append(item_dict)
    else:
        section_6_2_data = [{}]

    return section_6_2_data


def parse_otros_6_2(otros_search, column_headers):
    otros_6_2_data = []

    # parsing section for otros
    if len(otros_search) == 1:
        otros_string = otros_search[0]
        otros_list = otros_string.split("\n")
        otros_list = otros_list[1:]

        if len(otros_list) >= 64:
            # skip odd indexes (1, 3, 5, 7)
            count = 0
            count_restart = -2
            count_end = 20

            if len(otros_list) == 70:
                count_restart = -4

            item_dict = {}
            for item in otros_list:
                if count == count_end:
                    count = count_restart
                    otros_6_2_data.append(item_dict)
                    item_dict = {}
                if count == 0:
                    stripped_item = item.strip()
                    item_dict["product"] = stripped_item
                if count > 0 and count % 2 == 0:
                    col = column_headers[(count-2)//2]
                    stripped_item = item.strip()
                    item_dict[col] = stripped_item
                count += 1
        else:
            otros_6_2_data = [{}]
    else:
        otros_6_2_data = [{}]

    return otros_6_2_data

### Process the Whole File

In [ ]:
def process_files(source_directory, product_list,
                  translation_list, column_headers):
    dataset = []

    for process_file in os.listdir(source_directory):
        file_path = os.path.join(source_directory, process_file)

        # with statements automatically control the closing of files
        with open(file_path, "r") as file:
            contents = file.read()
            contents = contents.lower()

            section_start = "vi-2:"
            section_end = "vi-3:"
            # extract only the relevant section
            section_search = find_between(section_start, section_end, contents)

            otros_start = r"vi-2[\w\W]*otros[^\n]*\n"
            otros_search = find_between(otros_start, section_end, contents)

            interviewee = parse_interviewee(contents)
            date_range = parse_date_range(section_search)
            section_6_2_data = parse_section_6_2(section_search, product_list,
                                                 translation_list, column_headers)
            otros_6_2_data = parse_otros_6_2(otros_search, column_headers)

            data_dict = {}

            # check to make sure files contain the right amount of entries
            if len(section_6_2_data) <= 1:
                print(f"Unencountered Format: {process_file}\n"
                      + f"Length of section_data: {len(section_6_2_data)}\n")

            if len(otros_6_2_data) != 3 and len(otros_6_2_data) != 4:
                print(f"Unencountered Format: {process_file}\n"
                      + f"Length of otros_data: {len(otros_6_2_data)}\n")

            for row in section_6_2_data:
                data_dict = {}
                data_dict["filename"] = process_file
                data_dict["interviewee"] = interviewee
                data_dict["date_range"] = date_range

                for key, value in row.items():
                    data_dict[key] = value

                dataset.append(data_dict)

            for row in otros_6_2_data:
                data_dict = {}
                data_dict["filename"] = process_file
                data_dict["interviewee"] = interviewee
                data_dict["date_range"] = date_range

                for key, value in row.items():
                    data_dict[key] = value

                dataset.append(data_dict)

    # convert list to DataFrame
    raw_df = pd.DataFrame(data=dataset)

    return raw_df

## Transform

### Helper Functions

In [ ]:
def clean_date_range(dataf):
    replace_dict = {"agosto 2018 - agosto 2019": "august 2018-2019",
                    "septiembre 2018 – septiembre 2019": "september 2018-2019",
                    "octubre 2018 – octubre 2019": "october 2018-2019",
                    "octubre 2018 - octubre 2019": "october 2018-2019",
                    "octubre 2018 - octubre2019": "october 2018-2019",
                    "octubre 2018 – octubre  2019": "october 2018-2019",
                    "octubre  2018 – octubre 2019": "october 2018-2019",
                    "octubre  2018 – septiembre 2019": "october 2018 - september 2019"}
    dataf["date_range"] = dataf["date_range"].replace(replace_dict)

    return dataf


def clean_has_bought(dataf):
    map_dict = {"si": "yes",
                "ssi": "yes",
                "s1": "yes"}

    dataf["has_bought"] = dataf["has_bought"].replace(map_dict)

    return dataf


def clean_quantity(dataf):
    replace_dict = {"0": "",
                    "1/4": "0.25",
                    "1/2": "0.5",
                    "½": "0.5"}
    dataf["quantity"] = dataf["quantity"].replace(replace_dict)

    # replace thousands separator with nothing so that it can be converted to
    # float properly
    replace_filter = dataf["quantity"].str.contains(r"[.,]?\d{3}", na=False)
    replace_series = dataf["quantity"].str.replace(".", "")
    replace_series = replace_series.str.replace(",", "")

    dataf["quantity"] = dataf["quantity"].where(~replace_filter, replace_series)

    # handle decimal separator
    replace_filter_period = dataf["quantity"].str.contains(r",\d{1,2}",
                                                           na=False)
    replace_series_period = dataf["quantity"].str.replace(",", ".")
    dataf["quantity"] = dataf["quantity"].where(~replace_filter_period,
                                                replace_series_period)

    return dataf

### Pipeline Functions

In [ ]:
def start_pipeline(dataf):
    return dataf.copy()


def handle_null_data(dataf):
    # fill missing values with null
    dataf = dataf.replace("-", "")
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = ((isnull_sum >= 9)
                   | ((isnull_sum >= 6) & (dataf["has_bought"] == "no")))
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf


def clean_values(dataf):
    dataf = clean_date_range(dataf)
    dataf = clean_has_bought(dataf)
    dataf = clean_quantity(dataf)

    return dataf

### Create Raw and Cleaned DataFrame

In [ ]:
raw_df = process_files(source_directory, product_list,
                       translation_list, column_headers)

clean_df = (raw_df
            .pipe(start_pipeline)
            .pipe(clean_values)
            .pipe(handle_null_data))

## Other
### Functions to Check the Implementation

### Check Dropped Row

### Check Dropped Rows for Parsing Errors

## Load
### Export to CSV

In [ ]:
datasets_directory = "../datasets/"
filename = "section_6_2.csv"
file_path = os.path.join(datasets_directory, filename)

clean_df.to_csv(file_path, index=False, na_rep="null")

### Create Dropped DataFrame

***
# Testing Code
### View the Data

In [ ]:
clean_df.head(50)

### Check for Parsing Errors

In [ ]:
clean_df[clean_df["units"] == "2"]

In [ ]:
df_check = clean_df[clean_df["product"] == "vegetables_and_fruits"]

cols_to_check = df_check.columns
cols_to_drop = ["filename", "interviewee", "date_range"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(df_check[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

In [ ]:
cols_to_check = clean_df.columns
cols_to_drop = ["filename", "interviewee", "date_range"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(clean_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Check Dropped Rows
Another way to check is by opening the variable inspector and manually scrolling
through the `df_dropped` DataFrame. 

The code below tells us whether or not there are files that contain no information for this section. If raw file count is the same as clean file count, then no files were dropped.

In [ ]:
print("Raw File Count: {}".format(len(raw_df["filename"].value_counts())))
print("Clean File Count: {}".format(len(clean_df["filename"].value_counts())))

If there are files that are completely dropped, find those files and look at them to check for parsing errors.

In [ ]:
raw_file_names = raw_df["filename"].value_counts().index
clean_file_names = clean_df["filename"].value_counts().index

raw_file_names[~raw_file_names.isin(clean_file_names)]

Check the rows value counts to see if any rows are being dropped that should not be.

In [ ]:
cols_to_check = dropped_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(dropped_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Duplicate Rows

In [ ]:
df_columns = clean_df.columns[clean_df.columns != "filename"]
clean_df[clean_df.loc[:, df_columns].duplicated(keep=False)]